In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue May  6 12:23:49 2025

@author: ottocordero
"""
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.ndimage import gaussian_filter
from datetime import datetime

# ===========================================
# ------------ PARAMETERS -------------------
# ===========================================

GRID_SIZE = 400

# Initial densities
INITIAL_GREEN_DENSITY = 0.0005
INITIAL_RED_DENSITY = 0.0045

# Substrate and product concentrations
SUBSTRATE_INIT = 100.0
PRODUCT_INIT = 0.0

# Enzyme kinetics
KM = 0.05
ENZYME_INIT = 1.0
MAX_ENZYME_ACTIVITY = 0.1

# Growth kinetics (Monod-type)
GROWTH_RATE_PER_HOUR = np.log(2)
GROWTH_KM = 0.1

# Diffusion (Gaussian)
SUBSTRATE_DIFFUSION_SIGMA = 1.0
PRODUCT_DIFFUSION_SIGMA = 1.0

# ===========================================
# ------------ INITIALIZATION ----------------
# ===========================================

# Create timestamped output directory
timestamp = datetime.now().strftime("run_%Y-%m-%d_%H-%M-%S")
output_dir = os.path.join("simulation_outputs", timestamp)
os.makedirs(output_dir, exist_ok=True)

folders = {
    "images_cells": os.path.join(output_dir, "images_cells"),
    "csv_cells": os.path.join(output_dir, "csv_cells"),
    "images_product": os.path.join(output_dir, "images_product"),
    "csv_product": os.path.join(output_dir, "csv_product"),
}

for f in folders.values():
    os.makedirs(f, exist_ok=True)
    
# -----------------------------
# Save simulation parameters to README
# -----------------------------
readme_path = os.path.join(output_dir, "README.txt")

with open(readme_path, "w") as f:
    f.write("Simulation Parameters\n")
    f.write("=====================\n\n")
    f.write(f"GRID_SIZE = {GRID_SIZE}\n\n")

    f.write("Initial densities\n")
    f.write(f"  INITIAL_GREEN_DENSITY = {INITIAL_GREEN_DENSITY}\n")
    f.write(f"  INITIAL_RED_DENSITY = {INITIAL_RED_DENSITY}\n\n")

    f.write("Substrate and product concentrations\n")
    f.write(f"  SUBSTRATE_INIT = {SUBSTRATE_INIT}\n")
    f.write(f"  PRODUCT_INIT = {PRODUCT_INIT}\n\n")

    f.write("Enzyme kinetics\n")
    f.write(f"  KM = {KM}\n")
    f.write(f"  ENZYME_INIT = {ENZYME_INIT}\n")
    f.write(f"  MAX_ENZYME_ACTIVITY = {MAX_ENZYME_ACTIVITY}\n\n")

    f.write("Growth kinetics (Monod-type)\n")
    f.write(f"  GROWTH_RATE_PER_HOUR = {GROWTH_RATE_PER_HOUR}\n")
    f.write(f"  GROWTH_KM = {GROWTH_KM}\n\n")

    f.write("Diffusion (Gaussian filter sigmas)\n")
    f.write(f"  SUBSTRATE_DIFFUSION_SIGMA = {SUBSTRATE_DIFFUSION_SIGMA}\n")
    f.write(f"  PRODUCT_DIFFUSION_SIGMA = {PRODUCT_DIFFUSION_SIGMA}\n\n")

    f.write("Stopping criteria\n")
    f.write("  Simulation stops at cell density = 0.25\n")

print(f"README.txt written to {readme_path}")

# Initialize grids
grid = np.zeros((GRID_SIZE, GRID_SIZE), dtype=int)
substrate = np.full((GRID_SIZE, GRID_SIZE), SUBSTRATE_INIT, dtype=float)
product = np.full((GRID_SIZE, GRID_SIZE), PRODUCT_INIT, dtype=float)
enzyme_level = np.zeros((GRID_SIZE, GRID_SIZE), dtype=float)

green_cells = np.random.rand(GRID_SIZE, GRID_SIZE) < INITIAL_GREEN_DENSITY
red_cells = (np.random.rand(GRID_SIZE, GRID_SIZE) < INITIAL_RED_DENSITY) & (~green_cells)

grid[green_cells] = 2
grid[red_cells] = 1
enzyme_level[green_cells] = ENZYME_INIT

# Visualization scaling
product_global_max = 1e-6

# ---------- Save initial state ----------
frame = 0
pd.DataFrame(grid).to_csv(f"{folders['csv_cells']}/frame_{frame:04d}_cells.csv", index=False, header=False)
pd.DataFrame(product).to_csv(f"{folders['csv_product']}/frame_{frame:04d}_product.csv", index=False, header=False)

vis = np.zeros((GRID_SIZE, GRID_SIZE, 3), dtype=np.uint8)
vis[grid == 1] = [255, 0, 0]
vis[grid == 2] = [0, 255, 0]
plt.imsave(f"{folders['images_cells']}/frame_{frame:04d}_cells.png", vis)

plt.imsave(f"{folders['images_product']}/frame_{frame:04d}_product.png", product, cmap='Greys', vmin=0, vmax=product_global_max)

frame += 1

# ===========================================
# ------------ SIMULATION LOOP ---------------
# ===========================================

while np.mean(grid > 0) < 0.25:

    # Substrate breakdown
    vmax = enzyme_level * MAX_ENZYME_ACTIVITY
    rate = vmax * substrate / (KM + substrate)
    substrate -= rate
    product += rate
    substrate = np.clip(substrate, 0, None)

    # Diffusion
    product = gaussian_filter(product, sigma=PRODUCT_DIFFUSION_SIGMA)
    substrate = gaussian_filter(substrate, sigma=SUBSTRATE_DIFFUSION_SIGMA)

    product_global_max = max(product_global_max, product.max())

    # Cell growth
    new_grid = grid.copy()
    new_enzyme = enzyme_level.copy()

    for cell_type in [1, 2]:
        positions = np.argwhere(grid == cell_type)
        np.random.shuffle(positions)

        for i, j in positions:
            prod = product[i, j]
            grow_prob = GROWTH_RATE_PER_HOUR * prod / (GROWTH_KM + prod)

            if np.random.rand() < grow_prob:
                neighbors = [(i + di, j + dj) for di in [-1, 0, 1] for dj in [-1, 0, 1]
                             if not (di == 0 and dj == 0)]
                neighbors = [(ni % GRID_SIZE, nj % GRID_SIZE) for ni, nj in neighbors]
                empty_neighbors = [p for p in neighbors if grid[p] == 0]

                if empty_neighbors:
                    ni, nj = empty_neighbors[np.random.randint(len(empty_neighbors))]
                    new_grid[ni, nj] = cell_type

                    if cell_type == 2:
                        new_enzyme[i, j] /= 2
                        new_enzyme[ni, nj] = new_enzyme[i, j]

    grid = new_grid
    enzyme_level = new_enzyme

    # Save
    pd.DataFrame(grid).to_csv(f"{folders['csv_cells']}/frame_{frame:04d}_cells.csv", index=False, header=False)
    pd.DataFrame(product).to_csv(f"{folders['csv_product']}/frame_{frame:04d}_product.csv", index=False, header=False)

    vis = np.zeros((GRID_SIZE, GRID_SIZE, 3), dtype=np.uint8)
    vis[grid == 1] = [255, 0, 0]
    vis[grid == 2] = [0, 255, 0]
    plt.imsave(f"{folders['images_cells']}/frame_{frame:04d}_cells.png", vis)

    plt.imsave(f"{folders['images_product']}/frame_{frame:04d}_product.png", product, cmap='Greys', vmin=0, vmax=product_global_max)

    print(f"Frame {frame:04d} complete. Cell density: {np.mean(grid > 0):.3f}")
    frame += 1



README.txt written to simulation_outputs/run_2026-02-04_09-23-34/README.txt
Frame 0001 complete. Cell density: 0.005
Frame 0002 complete. Cell density: 0.005
Frame 0003 complete. Cell density: 0.005
Frame 0004 complete. Cell density: 0.005
Frame 0005 complete. Cell density: 0.005
Frame 0006 complete. Cell density: 0.005
Frame 0007 complete. Cell density: 0.005
Frame 0008 complete. Cell density: 0.005
Frame 0009 complete. Cell density: 0.006
Frame 0010 complete. Cell density: 0.006
Frame 0011 complete. Cell density: 0.006
Frame 0012 complete. Cell density: 0.006
Frame 0013 complete. Cell density: 0.007
Frame 0014 complete. Cell density: 0.007
Frame 0015 complete. Cell density: 0.008
Frame 0016 complete. Cell density: 0.008
Frame 0017 complete. Cell density: 0.009
Frame 0018 complete. Cell density: 0.009
Frame 0019 complete. Cell density: 0.010
Frame 0020 complete. Cell density: 0.011
Frame 0021 complete. Cell density: 0.012
Frame 0022 complete. Cell density: 0.013
Frame 0023 complete. C